# Video Understanding

In this example, we will generate a description of a video using `Qwen2-VL`, `Qwen2-5-VL`, `LLava`, and `Idefics3`, with more models coming soon.

This feature is currently in beta, may not work as expected.



## Install Dependencies

In [ ]:
!pip install -U mlx-vlm

## Import Dependencies

In [ ]:
from pprint import pprint
from mlx_vlm import load, generate
from mlx_vlm.video_generate import process_vision_info

import mlx.core as mx

In [ ]:
# Load the model and processor
model, processor = load("mlx-community/Qwen2.5-VL-7B-Instruct-4bit")

In [ ]:
# Messages containing a video and a text query
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "videos/fastmlx_local_ai_hub.mp4",
                "max_pixels": 360 * 360,
                "fps": 1.0,
            },
            {"type": "text", "text": "Describe this video."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)


In [ ]:
# Convert inputs to mlx arrays
input_ids = mx.array(inputs['input_ids'])
pixel_values = mx.array(inputs['pixel_values_videos'])
mask = mx.array(inputs['attention_mask'])
video_grid_thw = mx.array(inputs['video_grid_thw'])

kwargs = {
    "video_grid_thw": video_grid_thw,
}

In [ ]:
kwargs["video"] = "videos/fastmlx_local_ai_hub.mp4"
kwargs["input_ids"] = input_ids
kwargs["pixel_values"] = pixel_values
kwargs["mask"] = mask
response = generate(model, processor, prompt=text, temperature=0.7, max_tokens=100, **kwargs)

In [ ]:
pprint(response)


In [ ]:
# open video and play it
from ipywidgets import Video
Video.from_file("videos/fastmlx_local_ai_hub.mp4", width=320, height=240)